In [34]:
import pandas as pd
import hashlib
import os
import datetime as dt
import numpy as np
import pprint
import re
import email,smtplib, ssl
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [3]:
# e_ids=["ankitasngh21@gmail.com","jaijain96@gmail.com"]
# pwds =["ankita","jai"]
# d = {"email_id": e_ids,
#      "name": ["ankita","jai"],
#      "balance":[0.0,0.0],
#      "eid_hash":[hashlib.sha256(e_ids[0].encode("utf8")).hexdigest(),hashlib.sha256(e_ids[1].encode("utf8")).hexdigest()],
#      "pwd_hash":[hashlib.sha256(pwds[0].encode("utf8")).hexdigest(),hashlib.sha256(pwds[1].encode("utf8")).hexdigest()],
#      }
# df = pd.DataFrame(data=d)

In [6]:
# customer_data = {
#         e_ids[0] : {
#                       "pwd" : hashlib.sha256(pwds[0].encode("utf8")).hexdigest(),
#                       "balance" : 0.0
#          },
#         e_ids[1] : {
#                       "pwd" : hashlib.sha256(pwds[1].encode("utf8")).hexdigest(),
#                       "balance" : 0.0
#          }
# }
# np.save("customer_data.npy" ,customer_data)

{'ankitasngh21@gmail.com': {'pwd': '27c62a77d1e3f9b071d642bb5d368833254f697d5f91b7835e5102724272ba89', 'balance': 0.0}, 'jaijain96@gmail.com': {'pwd': 'bd3f131a260c9e6a5089eb46ca7ea1711102540805bf9b6ccdcc6f8c0995af38', 'balance': 0.0}}


In [2]:
customer_data = np.load("customer_data.npy" , allow_pickle = True).tolist()
pprint.pprint(customer_data)

{'ankitasngh21@gmail.com': {'balance': 995.0,
                            'pwd': '27c62a77d1e3f9b071d642bb5d368833254f697d5f91b7835e5102724272ba89'},
 'jaijain96@gmail.com': {'balance': 500.0,
                         'pwd': 'bd3f131a260c9e6a5089eb46ca7ea1711102540805bf9b6ccdcc6f8c0995af38'},
 'manager': {'balance': 0.0,
             'pwd': '8c6976e5b5410415bde908bd4dee15dfb167a9c873fc4bb8a81f6f2ab448a918'},
 'spark18manager@gmail.com': {'balance': 0.0,
                              'pwd': 'b9872adca3acd38ca5d82057aa7a6606a644b0425f291e57165e4947e4df11b4'},
 'xyz@gmail.com': {'balance': 0.0,
                   'pwd': '3608bca1e44ea6c4d268eb6db02260269892c0b42b86bbf1e77a6fa16c3c9282'}}


In [35]:
option =int(input("[1] Login\n[2] Register\n[3] Login as Manager\nPlease select anyone of the above options: "))
if option == 1:
    login_id = input("Please enter your login id : ")
    login_pwd = input("please enter your password : ")
    if (login_id in customer_data) and (customer_data[login_id]["pwd"] == hashlib.sha256(login_pwd.encode("utf8")).hexdigest()):
        print("loggedIn")
        trans_hist = load_transaction_history(login_id)
        do_transaction(customer_data,login_id,trans_hist)
        save_transaction_history(trans_hist,login_id)
    else:
        print("Invalid UserId or Password")
elif option == 2:
    login_id = input("Please enter your email id : ")
    login_pwd_0 = input("please enter your password : ")
    login_pwd_1 = input("confirm password : ")
    if login_pwd_0 == login_pwd_1 :
        customer_data[login_id] = {
            "balance": 0.0 ,
            "pwd" : hashlib.sha256(login_pwd_0.encode("utf8")).hexdigest()
        }
        print("You have been registered, current balance : ", customer_data[login_id]["balance"])
        np.save("customer_data.npy", customer_data)
    else:
        print("Passwords don't match")
elif option == 3:
    manager_id = input("Please enter manager id: ")
    manager_pwd = input("Please enter manager password: ")
    
    if (manager_id == "spark18manager@gmail.com") and (hashlib.sha256(manager_pwd.encode("utf8")).hexdigest() == customer_data[manager_id]["pwd"]):
        
        key_index = []
        index = 0
        for email_id in customer_data :
            if email_id != manager_id:
                print(index, email_id)
                key_index.append(email_id)
                index += 1
        selection = input("please select the indexes of customers you want to view transactions for, separated by spaces: ")
        selected_indices = np.unique(re.findall(r"\b\d+\b",selection))
        print("you have selected the following valid indices : ")
        for index in selected_indices:
            if int(index) < len(key_index) and int(index) > -1:
                print(index, key_index[int(index)])
                view_download_transaction(customer_data, key_index[int(index)],manager_id)
    else:
        print("Invalid Manager Password")
    

[1] Login
[2] Register
[3] Login as Manager
Please select anyone of the above options: 3
Please enter manager id: spark18manager@gmail.com
Please enter manager password: spark_18@ankita
0 ankitasngh21@gmail.com
1 jaijain96@gmail.com
2 xyz@gmail.com
please select the indexes of customers you want to view transactions for, separated by spaces: 0
you have selected the following valid indices : 
0 ankitasngh21@gmail.com
please enter the date time period you want to see transaction
from year: 2021
from month: 2
from day: 1
to year: 2021
to month: 4
to day: 1
transactions for time period  2021-02-01 00:00:00  to  2021-04-01 00:00:00
      date_time  balance transaction_type
0  1.616742e+09   1000.0          deposit
1  1.616742e+09    995.0        withdrawl
Do you want to mail yourself this transaction: y


In [24]:
def do_transaction(customer_data,login_id,trans_hist):
    transaction_type =int(input("[1] Deposit\n[2] Withdraw\n[3] Enquiry\nPlease select anyone of the above options"))
    print(transaction_type)
    if(transaction_type == 1):
        amt = int(input("Enter the amount you want to deposit: "))
        customer_data[login_id]["balance"] = customer_data[login_id]["balance"] + amt
        print("updated balance : ", customer_data[login_id]["balance"])
        np.save("customer_data.npy",customer_data)
        trans_hist["date_time"].append(dt.datetime.utcnow().timestamp())
        trans_hist["balance"].append(customer_data[login_id]["balance"])
        trans_hist["transaction_type"].append("deposit")
    elif(transaction_type ==2):
        amt = int(input("Enter the amount you want to withdraw: "))
        if amt > customer_data[login_id]["balance"]:
            print("amount withdrawn is greater than the current balance in account\ncurrent balance : ",
                 customer_data[login_id]["balance"])
        else:
            customer_data[login_id]["balance"] = customer_data[login_id]["balance"] - amt
            print("updated balance : ", customer_data[login_id]["balance"])
            np.save("customer_data.npy",customer_data)
            trans_hist["date_time"].append(dt.datetime.utcnow().timestamp())
            trans_hist["balance"].append(customer_data[login_id]["balance"])
            trans_hist["transaction_type"].append("withdrawl")
    elif(transaction_type == 3):
        print("Current Balance : ",customer_data[login_id]["balance"])

In [19]:
def load_transaction_history(login_id):
    foldername = "transaction_history"
    filename = hashlib.sha256(login_id.encode("utf8")).hexdigest() + ".csv"
    if  os.path.exists(os.path.join(foldername,filename) ) :
        return pd.read_csv(os.path.join(foldername,filename),index_col = False).to_dict("list")
    else:
        return {
            "date_time" : [],
            "balance": [],
            "transaction_type" : []
        }

In [20]:
def save_transaction_history(trans_hist,login_id):
    foldername = "transaction_history"
    filename = hashlib.sha256(login_id.encode("utf8")).hexdigest() + ".csv"
    pd.DataFrame(data=trans_hist).to_csv(os.path.join(foldername,filename),index=False)

In [21]:
def view_download_transaction(customer_data, login_id,manager_id):
    print("please enter the date time period you want to see transaction")
    fr_yy = int(input("from year: "))
    fr_mm = int(input("from month: "))
    fr_dd = int(input("from day: "))
    to_yy = int(input("to year: "))
    to_mm = int(input("to month: "))
    to_dd = int(input("to day: "))
    fr_date = dt.datetime(year = fr_yy, month = fr_mm, day = fr_dd)
    to_date = dt.datetime(year = to_yy, month = to_mm, day = to_dd)
    trans_hist = load_transaction_history(login_id)
    time_trans_hist = {
    "date_time":[],
    "balance":[],
    "transaction_type":[]
    }
    index = 0
    for timestamp in trans_hist["date_time"]:
        if timestamp >= fr_date.timestamp() and timestamp <= to_date.timestamp():
            time_trans_hist["date_time"].append(timestamp)
            time_trans_hist["balance"].append(trans_hist["balance"][index])
            time_trans_hist["transaction_type"].append(trans_hist["transaction_type"][index])
        index += 1
    print("transactions for time period ", fr_date," to ",to_date )
    trans_df = pd.DataFrame(data = time_trans_hist)
    print(trans_df)
    to_mail = input("Do you want to mail yourself this transaction: ")
    if to_mail == "y" or to_mail == "Y":
        mail_transaction(trans_df,login_id,fr_date,to_date, manager_id)
    

In [32]:
def mail_transaction(trans_df,login_id,fr_date,to_date, manager_id):
    subject = "Spark18 bank transaction history for "+ login_id
    body = "Please find attached transaction history for the billing period :" + str(fr_date) + " to " + str(to_date)
    sender_email = "spark18test@gmail.com"
    receiver_email = manager_id
    password = "spark_18@ankita"

    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = subject

    message.attach(MIMEText(body, "plain"))

    foldername = "transaction_history"
    filename = hashlib.sha256(login_id.encode("utf8")).hexdigest() + "transaction-history-" + str(fr_date.timestamp()) + \
    str(to_date.timestamp()) + ".csv"
    trans_df.to_csv(os.path.join(foldername, filename), index = False)

    with open(os.path.join(foldername,filename), "rb") as attachment:
        part = MIMEBase("application", "octet-stream")
        part.set_payload(attachment.read())

    # Encode file in ASCII characters to send by email    
    encoders.encode_base64(part)

    # Add header as key/value pair to attachment part
    part.add_header(
    "Content-Disposition",
    f"attachment; filename= transaction-history.csv",
    )

    # Add attachment to message and convert message to string
    message.attach(part)
    text = message.as_string()

    # Log in to server using secure context and send email
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, text)
    
    os.remove(os.path.join(foldername , filename))
    print("Mail has been successfully sent")

In [18]:
# #for Email
# import smtplib, ssl

# port = 465  # For SSL
# login_id = input("Please enter your login id")
# login_pwd = input("please enter your password and press enter here : ")
# if (login_id in customer_data) and (customer_data[login_id]["pwd"] == hashlib.sha256(login_pwd.encode("utf8")).hexdigest()):
#     context = ssl.create_default_context()

#     with smtplib.SMTP_SSL("smtp.gmail.com", port, context=context) as server:
#         server.login("spark18test@gmail.com","spark_18@ankita")
#         message = """\
#         Subject: Hi there

#         This message is sent from Python."""
#         server.sendmail("spark18test@gmail.com",login_id,message)
#         server.quit()

# else:
#     print("you have entered wrong id and password")


Please enter your login idankitasngh21@gmail.com
please enter your password and press enter here : ankita
